<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats2/blob/main/05_model_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# モデルの比較
* モデルを比較する方法を説明する。
* 参考資料
 * https://github.com/asuagar/statrethink-course-numpyro-2019/blob/main/statrethink_numpyro_w04.ipynb
* 例題として、事前分布の決め方が分析結果にどう影響するかを見つつ、モデルの比較を行なう。


## 準備

In [ ]:
!pip install arviz
!pip install numpyro

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import jax
import jax.numpy as jnp
from jax import random
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive

import arviz as az

%config InlineBackend.figure_format = 'retina'

plt.style.use("bmh")
rng_key = random.PRNGKey(0)
az.style.use("arviz-darkgrid")
numpyro.set_platform("cpu")

## 例題: Rosenthal and Jacobson (1968) の実験の結果を分析する
* 「統計モデリング1」で紹介だけした。
 * https://github.com/tomonari-masada/course2022-stats1/blob/main/06_normal_2.pdf
* データは原論文のものではなく、それに似せて作ったもの。
 * [STA 360/602: Bayesian Methods and Modern Statistics @ Duke University](http://www2.stat.duke.edu/~rcs46/bayes17.html)のModule 4より拝借。
* 明らかにしたいこと： 教師が期待をかけるか否かで学生の学修に影響があるか？
* つまり、$P(\mu_s > \mu_c | \mathbf{x}_s, \mathbf{x}_c)$を知りたい。
 * $\mathbf{x}_s \equiv \{ x_{s,1}, \ldots, x_{s,N_s}\}$: spurters（期待をかけられた学生たち）のIQ変化量データ群
 * $\mathbf{x}_c \equiv \{ x_{c,1}, \ldots, x_{c,N_c}\}$: controls（その他の学生たち）のIQ変化量データ群


### データ
* 数値は、実験開始直前からのIQの変化量を表す。

In [ ]:
x_s = [18, 40, 15, 17, 20, 44, 38]
x_c = [-4, 0, -19, 24, 19, 10, 5, 10,
       29, 13, -9, -8, 20, -1, 12, 21,
       -7, 14, 13, 20, 11, 16, 15, 27,
       23, 36, -33, 34, 13, 11, -19, 21,
       6, 25, 30,22, -28, 15, 26, -1, -2,
       43, 23, 22, 25, 16, 10, 29]

* データを一つのデータフレームにまとめる。

In [ ]:
df = pd.DataFrame(
    [{"group":"spurters", "value":val} for val in x_s]
    + 
    [{"group":"controls", "value":val} for val in x_c]
)

* ヒストグラムを描く。

In [ ]:
sns.histplot(data=df, x="value", bins=np.linspace(-35, 45, 33), hue="group")
plt.xlim(-35, 45)
plt.xlabel('Change in IQ Score')
plt.ylabel('count')
plt.title('Histogram of Change in IQ Scores');

### モデル
* 各グループの変化量は異なる正規分布に従うと仮定する。
 * spurtersの変化量のモデル $x_s \sim N(\mu_s, \lambda_s^{-1})$
 * controlsの変化量のモデル $x_c \sim N(\mu_c, \lambda_c^{-1})$
* 各グループの正規分布の平均パラメータは、同じ正規分布に従うと仮定する。
 * つまり $\mu_s, \mu_c \sim N(\mu_0, \sigma_0^2)$
* 各グループの正規分布の精度（分散の逆数）パラメータは、同じガンマ分布に従うと仮定する。
 * つまり $\lambda_s, \lambda_c \sim \text{Gam}(\alpha, \beta)$
* $N(\mu_0, \sigma_0^2)$と$\text{Gam}(\alpha, \beta)$が、事前分布。

### ハイパーパラメータの決め方
* 事前分布のパラメータをハイパーパラメータと呼ぶ。


* 精度パラメータが従うガンマ分布$\text{Gam}(\alpha, \beta)$については、[STA 360/602: Bayesian Methods and Modern Statistics @ Duke University](http://www2.stat.duke.edu/~rcs46/bayes17.html)のModule 4の決め方を踏襲する。
 * $\alpha$は$\frac{1}{2}$とする。
 * $\beta$は$10^2\alpha$とする。
* この設定は、以下の議論で固定しておく。

### 問題
* $\text{Gam}(\alpha, \beta)$のパラメータを上のように決めると、IQ変化量の分散ついて、どのように仮定していることになるか？
 * ヒント1： $\text{Gam}(\alpha, \beta)$がモデリングしているのは、精度、つまり、分散の逆数であることに注意しよう。
 * ヒント2： $\text{Gam}(\alpha, \beta)$に従う確率変数の値の平均は、$\frac{\alpha}{\beta}$である。

### 事前分布 (1)
* 平均パラメータが従う正規分布$\mu \sim N(\mu_0, \sigma_0^2)$について、まずは以下のように設定してみる。
 * 別の設定については、後で議論する。



#### 事前分布の平均パラメータ$\mu_0$の設定
* $\mu_0 = 0$とする。
 * なぜなら、IQの変化量の平均が、増えるのか、減るのか、全く分からないから。

#### 事前分布の分散パラメータ$\sigma_0^2$の設定
* $\sigma_0 = 1$とする。
 * これは、ざっくり言うと、どういうお気持ちを表しているか？


* 考察をしやすくするために、観測データが従う正規分布$N(\mu, \sigma^2)$の分散$\sigma^2$を固定する。
* すると、$\mu$の事後分布の分散は$(\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2})^{-1}$となる。
 * $n$は観測データの個数。
* $\sigma_0$を大きくすると、$n$個の観測データで決まる$\frac{n}{\sigma^2}$の項のほうが支配的となる。
* $\sigma_0$を小さくすると、$\frac{1}{\sigma_0^2}$の項のほうが支配的となる。
* $\sigma_0=1$という仮定は、ざっくり言って、これらの中間。
* つまり、IQ変化量の平均$\mu$がどのくらいバラつくかという不確かさについて、（非常にざっくり言って）事前分布が観測データ一個分と同程度の寄与をする、と仮定している。
 * 観測データ0個分でもなく、全観測データ以上分でもない、ということ。

#### モデルの実装

In [ ]:
df[df.group=="spurters"].value.values

In [ ]:
def model(mu_0=0, sd_0=1, alpha=0.5, x_s=None, x_c=None):
  lambda_s = numpyro.sample("lambda_s", dist.Gamma(alpha, 100*alpha))
  lambda_c = numpyro.sample("lambda_c", dist.Gamma(alpha, 100*alpha))
  mu_s = numpyro.sample("mu_s", dist.Normal(mu_0, sd_0))
  mu_c = numpyro.sample("mu_c", dist.Normal(mu_0, sd_0))
  obs_s = numpyro.sample("obs_s", dist.Normal(mu_s, jnp.sqrt(1/lambda_s)), obs=x_s)
  obs_c = numpyro.sample("obs_c", dist.Normal(mu_c, jnp.sqrt(1/lambda_c)), obs=x_c)

#### MCMC

In [ ]:
mu_0 = 0.0
sd_0 = 1.0
alpha = 0.5

rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)
mcmc = MCMC(NUTS(model), num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(
    rng_key_,
    mu_0=mu_0, sd_0=sd_0, alpha=alpha,
    x_s=df[df.group=="spurters"].value.values,
    x_c=df[df.group=="controls"].value.values,
)

#### サンプルのチェック

In [ ]:
mcmc.print_summary()

In [ ]:
idata_1 = az.from_numpyro(mcmc)
az.plot_trace(idata_1);

In [ ]:
az.plot_autocorr(idata_1, combined=True, figsize=(16,3));

* この設定の下で、$p(\mu_s > \mu_c | \mathbf{x}_s, \mathbf{x}_c)$を求めてみる。

In [ ]:
posterior = idata_1.posterior
n_samples = posterior.dims['chain'] * posterior.dims['draw']
(posterior.mu_s > posterior.mu_c).sum().data / n_samples

* $\mu_s > \mu_c$の確率が0.2前後
 * 学生に期待をかけないほうがいい？

### 事前分布 (2)
* ハイパーパラメータの決め方を考え直す。

#### 事前分布の平均パラメータ$\mu_0$の設定
* $\mu_0$を観測データの平均値とする。
 * なぜなら、一定期間教育を受ければ、そもそもIQは増えるものであるから。
 * この意味で、$\mu_0 = 0$という設定は、変だった。
 * しかし、どのくらい増えるものかについて、観測データ以外に手がかりがない。
 * そのため、観測データの単純平均を使うことにする。

#### 事前分布の分散パラメータ$\sigma_0^2$の設定
* $\sigma_0$は100ぐらいとする。
 * つまり、IQ変化量の平均がどのくらいバラつくかという不確かさについては、観測データに決めてもらう、ということ。

#### MCMC

In [ ]:
mu_0 = np.array(x_s + x_c).mean()
sd_0 = 100.0
alpha = 0.5

rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)
mcmc = MCMC(NUTS(model), num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(
    rng_key_,
    mu_0=mu_0, sd_0=sd_0, alpha=alpha,
    x_s=df[df.group=="spurters"].value.values,
    x_c=df[df.group=="controls"].value.values,
)

In [ ]:
mcmc.print_summary()

#### 4.4.5 サンプルのチェック

In [ ]:
idata_2 = az.from_numpyro(mcmc)
az.plot_trace(idata_2);

In [ ]:
az.plot_autocorr(idata_2, combined=True, figsize=(16,3));

* この設定の下で、$p(\mu_s > \mu_c | \mathbf{x}_s, \mathbf{x}_c)$を求めてみる。

In [ ]:
posterior = idata_2.posterior
n_samples = posterior.dims['chain'] * posterior.dims['draw']
(posterior.mu_s > posterior.mu_c).sum().data / n_samples

* 先ほどと真逆の結果。

### モデルの比較
* 上の二通りの事前分布の設定方法のうち、どちらが良いかを調べてみる。
* ArviZの`compare`関数を使う。
 * https://python.arviz.org/en/stable/api/generated/arviz.compare.html
* 理論的な説明は https://arxiv.org/abs/1507.04544 を参照。

* 上の論文によると、elpdは以下のように定義される評価値である。
$$\begin{align}
\text{elpd}_{\text{loo}} = \sum_{i=1}^N \log p( x_i | x_{- i} ) 
\end{align}$$
where
$$\begin{align}
p( x_i | x_{- i} ) = \int p(x_i | \theta) p( \theta | x_{-i}) d\theta
\end{align}$$
* この計算を、事後分布からのサンプルを使って、近似的に行なっている。


* まず、spurtersのデータの予測確率を使って比較する。

In [ ]:
az.compare({'model_1': idata_1, 'model_2': idata_2}, var_name="obs_s", scale='deviance')

* 次に、controlsのデータの予測確率を使って比較する。

In [ ]:
az.compare({'model_1': idata_1, 'model_2': idata_2}, var_name="obs_c", scale='deviance')

* controlsのほうがelpd_looの値が大きく出ているのは、controlsのほうがデータ数が多いから。

In [ ]:
print(f"We have {len(x_s)} spurters and {len(x_c)} controls.")

* いずれも、二つ目のハイパーパラメータの決め方のほうが良い、という結果である。
* ということは、二つ目の設定の下での$p(\mu_s > \mu_c | \mathbf{x}_s, \mathbf{x}_c)$の値を採用するほうが良い、ということだろう。

# 課題4
* あなたなら、どのように事前分布を決める？
* その決め方にしたがって分析をおこない、$p(\mu_s > \mu_c | \mathbf{x}_s, \mathbf{x}_c)$を、上と同様にして求めよう。
* さらに、上の二つのケースと比較してみよう。